In [64]:
import os
import time
import numpy as np
import openai
from importlib import reload
import sys
from openai import OpenAI

import baseDocumentParser
reload(baseDocumentParser)
from baseDocumentParser import *

import basePromptDetails
reload(basePromptDetails)
from basePromptDetails import promptDetails

from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient

documentParser=documentParser()
promptDetails=promptDetails()

In [45]:
import os
os.getcwd()

'/Users/ashwini/Desktop/stuff/notebooks/adi'

#### path of sample invoice document

In [15]:
file_name="./uploads/Sample_Invoice.pdf"

#### Specify document details: document type, short description, and data elements to be extracted

In [16]:
document_type_name="Invoice"
document_type_description="""An invoice is a formal document issued by a seller to a buyer, detailing a transaction. It outlines the goods or services provided, their quantities, unit prices, and the total amount owed. Key data elements in an invoice include the invoice number, issue date, buyer and seller contact details, item descriptions, unit prices, taxes, discounts, payment terms, and the total payable amount. The primary purpose of an invoice is to request payment, provide a record of the transaction, and facilitate accounting and tax reporting for both parties involved in the business transaction."""
data_elements="""Invoice Number, Invoice Date, Supplier Name, Buyer Name, Item Description, Quantity, Unit Price, Total Amount, Tax Amount, Payment Terms"""

In [17]:
data_elements_str="\n".join(data_elements.split(", "))
data_elements_str

'Invoice Number\nInvoice Date\nSupplier Name\nBuyer Name\nItem Description\nQuantity\nUnit Price\nTotal Amount\nTax Amount\nPayment Terms'

#### Call ADI parser to extract document contents

In [20]:
file_data = open(file_name, "rb")
model_id="prebuilt-invoice"
adi_response=documentParser.get_azure_parsed_contents(file_data, model_id)

In [8]:
### Openai call to adi parsed document
### need metaprompt, parsed ADI content

In [21]:
table_level_data=documentParser.get_table_contents(adi_response)
document_level_data=documentParser.get_document_contents(adi_response)

In [22]:
doc_json = {"paragraph_data": document_level_data, "table_data" : table_level_data}

In [ ]:
#### OpenAI api call to parse the document level data elements

In [24]:
prompts_details=promptDetails.get_metaprompt()
system_prompt_message=prompts_details['system_prompt']
model="gpt-4o"
response_format="json_object"

user_prompt_message = f"""**Document Type Details:** 
                            \n Document Type: {document_type_name} 
                            \n Document Description:{document_type_description}
                            \n**Data Elements for Extraction:** \n{data_elements_str} 
                            \n\n**Document Content:** \n{json.dumps(doc_json)}"""

messages = [
    {"role": "system", "content": "You are a data extractor expert for various documents types and various content type" + system_prompt_message},
    {"role": "user", "content": user_prompt_message},
]

In [27]:
status, openai_response=documentParser.get_openai_formatted_response(messages=messages)
response_json=documentParser.json_post_process(openai_response)

#### Parse the document details to paragraph and table dataframes

In [46]:
table_col_names_str="\n".join(documentParser.get_table_col_names(doc_json))

message_col_names=promptDetails.get_openai_column_mapped_names_message(document_type_name, document_type_description, data_elements_str, table_col_names_str)
status, openai_response_col_names=documentParser.get_openai_formatted_response(messages=message_col_names)

table_cols_to_data_elements_dict=documentParser.json_post_process(openai_response_col_names)

table_cols_to_data_elements_dict

{'Quantity': 'Quantity',
 'Items': 'n/a',
 'Units': 'n/a',
 'Descri': 'Item Description',
 'Discount %': 'n/a',
 'Taxable': 'Tax Amount',
 'Unit Price': 'Unit Price',
 'Total': 'Total Amount'}

In [51]:
para_dict=documentParser.get_para_dict(response_json)
documentParser.get_para_df(para_dict)

,data element,data value
0,Supplier Name,Spa Supplies
1,Buyer Name,Olympic World Hotel.
2,Invoice Date,2020-03-02
3,Invoice Number,PLC0920
4,Total Amount,"$354,967.00"


In [52]:
table_list=documentParser.get_table_dict(response_json)

table_cols=[x[0] for x in table_list]

documentParser.get_table_col_names(doc_json)

['Quantity',
 'Items',
 'Units',
 'Descri',
 'Discount %',
 'Taxable',
 'Unit Price',
 'Total']

In [65]:
data_elements_order={x.strip():i for i,x in enumerate(data_elements.split(","))}
table_df=documentParser.get_table_dataframe(doc_json, table_cols_to_data_elements_dict, data_elements_order)
table_df

,Item Description,Quantity,Unit Price,Total Amount,Tax Amount
0,Massage tables,14,"$2,300","$32,200",N/A
1,Towel heaters,14,$768,"$10,752",N/A
2,Entry-way chairs,7,"$1,136","$7,952",N/A
3,Wax heaters,14,$987,"$13,818",N/A
4,Spa loungers,16,$979,"$15,616",N/A
5,Hot tub,4,"$50,613","$202,453",N/A
6,Pedicure chair,12,"$3,456","$41,472",N/A
7,Manicure and Pedicure dryer,4,"$1,456","$5,824",N/A
8,Manicure tables,10,"$2,488","$24,880",N/A ;
